# Holodeck Populations

This notebooks examines holodeck populations drawn from fits to GW data.  
The data is constructed using the `gen_holodeck_pops.py` script.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import holodeck as holo
from holodeck.librarian import posterior_populations
from holodeck.constants import MSOL, YR

# Load data

### from save file

(generated by `holodeck/librarian/posterior_populations.py`)

In [ ]:
# Specify filename to load (produced from `gen_holodeck_pops.py` script)
fname = "./output/t20.0yr_nf30_nr100_nl10_ML_0000.npz"
# Load data
data = np.load(fname)

### generate from posterior chains

In [ ]:
pars = posterior_populations.get_maxlike_pars_from_chains()
data, classes = posterior_populations.load_population_for_pars(pars)

## Print basic information about data

In [ ]:
print("data keys = ", list(data.keys()))

print("Name of parameter space used = ", data['pspace_name'])
number = data['number']
*grid_shape, nfreqs = number.shape
print(f"Shape of SAM grid = {grid_shape} = (M, Q, Z); with {nfreqs} frequency bins")

hc_ss = data['hc_ss']
nfreqs, nreals, nloudest = hc_ss.shape
print(f"Populations generated for {nreals} realizations, with {nloudest} loudest binaries per frequency bin")

## Number of binaries in the Universe

In [ ]:
# This is the expectation value for the number of binaries in each bin
# Shape (M, Q, Z, F) which is  (total-mass, mass-ratio, redshift, frequency)
number = data['number']
print(f"Total number of binaries = {number.sum():.4e}")
params = ['mtot', 'mrat', 'redz', 'fobs_orb']
units = [MSOL, 1.0, 1.0, 1/YR]

# Plot number of binaries vs. each parameter of grid
fig, axes = holo.plot.figax(figsize=[12, 3], ncols=4)
for ii, ax in enumerate(axes):
    # Load the bin edges for the parameter of interest (ii)
    pname = params[ii] + "_edges"
    # note that the number of bin 'edges' is +1 the number of bins
    edges = data[pname] / units[ii]
    # Sum over all of the dimensions except the dimension of interest
    marg = np.arange(4).tolist()
    marg.pop(ii)
    num = number.sum(axis=tuple(marg))
    # plot histogram
    holo.plot.draw_hist_steps(ax, edges, num)
    ax.set_xlabel(f"{params[ii]}")
    # Set minimum yaxis lower-limit to 1e-2
    ylim = list(ax.get_ylim())
    ylim[0] = np.clip(ylim[0], 1.0e-2, None)
    ax.set_ylim(ylim)

plt.show()


## Gravitational-Wave strains

In [ ]:
# Number of realizations to plot
plot_num_reals = 5
# Number of loudest-binaries-per-bin to plot
plot_num_loudest = 1

# ---- Load data

# Frequency bin edges, in units of [1/sec], for orbital frequencies
fobs_orb_edges = data['fobs_orb_edges']
# convert to GW frequency centers (assuming circular orbits)
fobs_orb_cents = 0.5 * (fobs_orb_edges[1:] + fobs_orb_edges[:-1])
fobs_gw_cents = 2.0 * fobs_orb_cents * YR

# Characteristic strains of the L loudest binaries in each of F frequency bins for each of R realizations
# shape: (F, R, L)
hc_ss = data['hc_ss']
nfreqs, nreals, nloudest = hc_ss.shape
# Characteristic strain of all *other* binaries (i.e. All minus L)
# shape: (F, R)
hc_bg = data['hc_bg']

# can't plot more elements than we have in the data
plot_num_reals = np.minimum(plot_num_reals, nreals)
plot_num_loudest = np.minimum(plot_num_loudest, nloudest)

# ---- Plot GW signals

fig, ax = holo.plot.figax()
for rr in range(plot_num_reals):
    # this is the combined GWs from all binaries except `L` loudest in each frequency bin
    bg = hc_bg[:, rr]
    # add back in the strains from the loudest binaries that we aren't plotting
    if nloudest > plot_num_loudest:
        fg = hc_ss[:, rr, plot_num_loudest:]
        # Add in quadrature, summing over however many loudest binaries remain
        bg = np.sqrt(bg**2 + np.sum(fg**2, axis=-1))

    hh, = ax.plot(fobs_gw_cents, bg, alpha=0.75)
    for ll in range(plot_num_loudest):
        ax.scatter(fobs_gw_cents, hc_ss[:, rr, ll], color=hh.get_color(), s=10, alpha=0.5)

# Plot a -2/3 power-law of amplitude 1e-15 for reference
hc_plaw = 1.0e-15 * np.power(fobs_gw_cents, -2.0/3.0)
ax.plot(fobs_gw_cents, hc_plaw, 'k--', alpha=0.5, lw=1.0)

plt.show()


## Properties of loudest single binaries

In [ ]:
# This array contains the binary parameters for each of the 'L' loudest binaries in each frequency bin
# shape: (P, F, R, L) = (parameters, frequencies, realizations, loudest)
# the P=4 parameters are {total mass, mass ratio, initial-redshift, final-redshift}
# initial-redshift refers to the time of galaxy merger, final redshift refers to the redshift at this frequency
sspar = data['sspar']

par_names = ["total mass", "mass ratio", "initial redshift", "final redshift"]
units = [MSOL, 1.0, 1.0, 1.0]

# specific frequency bins to plot
plot_fbins = [0, nfreqs//2, nfreqs-1]
colors = plt.get_cmap('Oranges')(np.linspace(0.5, 1.0, len(plot_fbins)))

fig, axes = holo.plot.figax(figsize=[12, 3], ncols=4)
for ii, ax in enumerate(axes):
    ax.set(xlabel=par_names[ii])

    vals = sspar[ii].flatten() / units[ii]
    hist, edges = np.histogram(np.log10(vals), bins=20, density=False)
    holo.plot.draw_hist_steps(ax, 10.0 ** edges, hist)

    for jj, ff in enumerate(plot_fbins):
        vals = sspar[ii, ff].flatten() / units[ii]
        hist, edges = np.histogram(np.log10(vals), bins=10, density=False)
        lab = f"{ff}" if ii == 1 else None
        holo.plot.draw_hist_steps(ax, 10.0 ** edges, hist, alpha=0.5, lw=1.0, color=colors[jj], label=lab)

axes[1].legend(title='frequency bin', fontsize=8, title_fontsize=8)
plt.show()

